## Image Alignment Based On OpenCV
- Ref : https://www.pyimagesearch.com/2020/08/31/image-alignment-and-registration-with-opencv/

### Flow
1. Keypoint Detection on two input images
2. Estimate homography matrix
3. Apply perspective warping

### Import

In [14]:
import cv2
import numpy as np

from matplotlib import pyplot as plt

### Parameters

In [35]:
TEMPLATE_FPATH = 'template.png'
SAMPLE_FPATH = 'scan_01.jpg'
RESIZE_RATIO = 0.5

MAX_KEYPOINT_NUM = 500
KEEP_PERC = 0.2

### Load Image

In [36]:
# Read Image
img_sample = cv2.imread(SAMPLE_FPATH)
img_template = cv2.imread(TEMPLATE_FPATH)

In [37]:
# Resize (Only to Make Image Size Smaller)
img_sample = cv2.resize(img_sample, (int(img_sample.shape[1] * RESIZE_RATIO), int(img_sample.shape[0] * RESIZE_RATIO)))
img_template = cv2.resize(img_template, (int(img_template.shape[1] * RESIZE_RATIO), int(img_template.shape[0] * RESIZE_RATIO)))

### Preprocessing

In [38]:
# BGR To Grayscale
gray_sample = cv2.cvtColor(img_sample, cv2.COLOR_BGR2GRAY)
gray_template = cv2.cvtColor(img_template, cv2.COLOR_BGR2GRAY)

### Keypoint Detection and Description (Oriented Fast and Rotated BRIEF, ORB)

In [39]:
# Initialize ORB keypoint detector and descriptor
orb = cv2.ORB_create(MAX_KEYPOINT_NUM)

In [40]:
# Keypoint Detect / Description (None for no image mask)
# Keypoint -- List of KeyPoint objects
# Description -- Numpy array (#Keypoints, 32) (256-bit descriptor)
kps_s, desc_s = orb.detectAndCompute(gray_sample, None)
kps_t, desc_t = orb.detectAndCompute(gray_template, None)

### Keypoint Matching

In [41]:
# Initialize Keypoint Descriptor Matcher
# Use Hamming distance due to ORB is a binary descriptor
matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)

In [42]:
# Keypoint Matching
# Matches -- List of DMatch objects
matches = matcher.match(desc_s, desc_t, None)

In [43]:
# Keep only top matching result
matches = sorted(matches, key=lambda x: x.distance)
matches = matches[:int(len(matches) * KEEP_PERC)]

### Estimate Homography Matrix

In [45]:
# Get Matched Keypoint Coodinates
pts_s = np.zeros((len(matches), 2), dtype=np.float)
pts_t = np.zeros((len(matches), 2), dtype=np.float)

for mid, match in enumerate(matches):
    pts_s[mid] = kps_s[match.queryIdx].pt
    pts_t[mid] = kps_t[match.trainIdx].pt    

In [50]:
# Compute Homography Matrix based on RANSAC
H, mask = cv2.findHomography(pts_s, pts_t, cv2.RANSAC)

### Perspective Warping

In [52]:
img_aligned = cv2.warpPerspective(img_sample, H, (img_template.shape[1], img_template.shape[0]))

### Visualization

In [ ]:
# Result -- Top-10 Keypoint Matching
img_match_vis = cv2.drawMatches(img_sample, kps_s, img_template, kps_t, matches[:10], None)

fig = plt.figure(figsize=(10, 10))
plt.imshow(img_match_vis)

In [ ]:
# Result -- Alignment
fig = plt.figure(figsize=(10, 24))
plt.imshow(np.hstack([img_template, img_aligned]))